In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv

load_dotenv()

spark = SparkSession.builder \
        .remote("sc://localhost:15002") \
        .appName("HybridDataPipelineSpark") \
        .getOrCreate()

In [ ]:
from jdbc import read_table

df_customer = read_table(spark, "SalesLT.Customer")
df_sales_order_header = read_table(spark, "SalesLT.SalesOrderHeader")
df_sales_order_detail = read_table(spark, "SalesLT.SalesOrderDetail")
df_product = read_table(spark, "SalesLT.Product")
df_product_category = read_table(spark, "SalesLT.ProductCategory")

In [31]:
df_customer.write.mode("overwrite").parquet("/data/bronze/customer")
df_sales_order_header.write.mode("overwrite").parquet("/data/bronze/sales_order_header")
df_sales_order_detail.write.mode("overwrite").parquet("/data/bronze/sales_order_detail")
df_product.write.mode("overwrite").parquet("/data/bronze/product")
df_product_category.write.mode("overwrite").parquet("/data/bronze/product_category")

In [23]:
customer_bronze = spark.read.parquet("/data/bronze/customer")
customer_silver = customer_bronze.select("CustomerID", "FirstName", "LastName").dropDuplicates(["CustomerID"])

customer_silver.write.mode("overwrite").parquet("/data/silver/customer")

In [ ]:
product_bronze = spark.read.parquet("/data/bronze/product")

product_bronze = product_bronze.select("ProductID", "Name", "Color", "StandardCost", "ListPrice", "ProductCategoryID", "SellStartDate", "SellEndDate", "DiscontinuedDate").dropDuplicates(["ProductID"])

+------------+-----+
|       Color|count|
+------------+-----+
|        NULL|   50|
|       Multi|    8|
|      Silver|   36|
|        Blue|   26|
|       White|    4|
|       Black|   89|
|      Yellow|   36|
|         Red|   38|
|        Grey|    1|
|Silver/Black|    7|
+------------+-----+

